<a href="https://colab.research.google.com/github/etodasha/PRA-M2.875-2020.2/blob/main/PRA_DL_deeper_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes


In [ ]:
!kaggle datasets download "jordidelatorreuoc/kaggle-plant-pathology-2021-modificat"

100% 966M/970M [00:10<00:00, 111MB/s]
100% 970M/970M [00:10<00:00, 97.2MB/s]


In [ ]:
!unzip kaggle-plant-pathology-2021-modificat.zip

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import keras 
import os
import matplotlib.pylab as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, ZeroPadding2D, GlobalAveragePooling2D,BatchNormalization, AveragePooling2D, Input
from keras.optimizers import Adam, SGD,RMSprop,Adadelta,Adagrad,Adamax,Nadam,Ftrl
from keras.layers.experimental.preprocessing import Resizing
from keras.callbacks import *


In [ ]:
datagen_train = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8,1.2],
    zoom_range=[0.5,1.2],
    rotation_range=20,
    rescale=1./255,
    validation_split=0.2) 

train_generator = datagen_train.flow_from_directory( 
    'train',
    subset='training')

validation_generator = datagen_train.flow_from_directory(
    'train',
    subset='validation'
    ,shuffle=False)



Found 7800 images belonging to 6 classes.
Found 1950 images belonging to 6 classes.


In [ ]:
from keras import backend as K

def custom_recall(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = TP / (Positives+K.epsilon())
  return recall 

def custom_precision(y_true, y_pred):
  TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = TP / (Pred_Positives+K.epsilon())
  return precision

def custom_f1(y_true, y_pred):    
    precision, recall = custom_precision(y_true, y_pred), custom_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

deeper_model = Sequential()
deeper_model.add(keras.Input(shape=(332, 498, 3)))

deeper_model.add(Resizing(64,64))

deeper_model.add(Conv2D(32, 3, padding="same", strides=(1, 1), activation="relu"))
deeper_model.add(MaxPool2D(pool_size=(2, 2)))

deeper_model.add(Conv2D(64, 3, padding="same", strides=(1, 1), activation="relu"))
deeper_model.add(MaxPool2D(pool_size=(2, 2)))

deeper_model.add(Conv2D(128, 3, padding="same", strides=(1, 1), activation="relu"))
deeper_model.add(MaxPool2D(pool_size=(2, 2)))

deeper_model.add(Conv2D(256, 3, padding="same", strides=(1, 1), activation="relu"))
deeper_model.add(MaxPool2D(pool_size=(2, 2)))

deeper_model.add(Conv2D(728, 3, padding="same", strides=(1, 1), activation="relu"))
deeper_model.add(MaxPool2D(pool_size=(2, 2)))

deeper_model.add(GlobalAveragePooling2D())

deeper_model.add(Dense(1024,activation="relu"))
deeper_model.add(Dropout(0.4))

deeper_model.add(Dense(6, activation="softmax"))

deeper_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0

In [ ]:
deeper_model.compile(optimizer = Adam(learning_rate=0.0001), 
              loss = "categorical_crossentropy", 
              metrics = ['accuracy',custom_f1,custom_recall,custom_precision])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_custom_f1', mode='max', patience=15, 
                                               restore_best_weights=True, verbose=2)

filepath = "/content/gdrive/My Drive/epochs:{epoch:03d}-val_f1:{val_custom_f1:.3f}.hdf5"

checkpoint_callback = ModelCheckpoint(filepath= filepath, monitor='val_custom_f1', mode='max',
                                                         save_weights_only=True, save_best_only=True)


deeper_history = deeper_model.fit(train_generator,
                              validation_data = validation_generator,
                              epochs=100,
                              batch_size=32,
                              callbacks=[early_stopping, checkpoint_callback]) 

Epoch 1/100
244/244 [==============================] - 159s 646ms/step - loss: 1.3488 - accuracy: 0.4987 - custom_f1: 0.3994 - custom_recall: 0.2935 - custom_precision: 0.6591 - val_loss: 1.3075 - val_accuracy: 0.5144 - val_custom_f1: 0.3510 - val_custom_recall: 0.2772 - val_custom_precision: 0.5353
Epoch 2/100
244/244 [==============================] - 157s 642ms/step - loss: 1.3266 - accuracy: 0.5098 - custom_f1: 0.3843 - custom_recall: 0.2795 - custom_precision: 0.6477 - val_loss: 1.2925 - val_accuracy: 0.5410 - val_custom_f1: 0.3388 - val_custom_recall: 0.2520 - val_custom_precision: 0.5672
Epoch 3/100
244/244 [==============================] - 156s 642ms/step - loss: 1.3200 - accuracy: 0.5108 - custom_f1: 0.3771 - custom_recall: 0.2697 - custom_precision: 0.6688 - val_loss: 1.2928 - val_accuracy: 0.5210 - val_custom_f1: 0.4015 - val_custom_recall: 0.3397 - val_custom_precision: 0.5165
Epoch 4/100
244/244 [==============================] - 157s 643ms/step - loss: 1.2772 - accuracy:

In [ ]:
deeper_model.compile(optimizer = Adam(learning_rate=0.00001), # change learning rate 
              loss = "categorical_crossentropy", 
              metrics = ['accuracy',custom_f1,custom_recall,custom_precision])

deeper_model.load_weights('/content/gdrive/My Drive/epochs:100-val_f1:0.834.hdf5')

early_stopping = keras.callbacks.EarlyStopping(monitor='val_custom_f1', mode='max', patience=25, 
                                               restore_best_weights=True, verbose=2)

filepath = "/content/gdrive/My Drive/epochs:{epoch:03d}-val_f1:{val_custom_f1:.3f}.hdf5"

checkpoint_callback = ModelCheckpoint(filepath= filepath, monitor='val_custom_f1', mode='max',
                                                         save_weights_only=True, save_best_only=True)


deeper_history = deeper_model.fit(train_generator,
                              validation_data = validation_generator,
                              initial_epoch=100,
                              epochs=200,
                              batch_size=32,
                              callbacks=[early_stopping, checkpoint_callback]) 

Epoch 101/200
244/244 [==============================] - 159s 646ms/step - loss: 0.3725 - accuracy: 0.8711 - custom_f1: 0.8668 - custom_recall: 0.8395 - custom_precision: 0.8969 - val_loss: 0.4609 - val_accuracy: 0.8462 - val_custom_f1: 0.8452 - val_custom_recall: 0.8204 - val_custom_precision: 0.8732
Epoch 102/200
244/244 [==============================] - 156s 640ms/step - loss: 0.3477 - accuracy: 0.8823 - custom_f1: 0.8826 - custom_recall: 0.8594 - custom_precision: 0.9080 - val_loss: 0.4776 - val_accuracy: 0.8318 - val_custom_f1: 0.8326 - val_custom_recall: 0.8095 - val_custom_precision: 0.8581
Epoch 103/200
244/244 [==============================] - 156s 640ms/step - loss: 0.3599 - accuracy: 0.8796 - custom_f1: 0.8775 - custom_recall: 0.8541 - custom_precision: 0.9030 - val_loss: 0.4351 - val_accuracy: 0.8451 - val_custom_f1: 0.8449 - val_custom_recall: 0.8189 - val_custom_precision: 0.8749
Epoch 104/200
244/244 [==============================] - 156s 639ms/step - loss: 0.3432 - a

In [ ]:
deeper_model.compile(optimizer = Adam(learning_rate=0.00001), # change learning rate 
              loss = "categorical_crossentropy", 
              metrics = ['accuracy',custom_f1,custom_recall,custom_precision])

deeper_model.load_weights('/content/gdrive/My Drive/epochs:100-val_f1:0.834.hdf5')

early_stopping = keras.callbacks.EarlyStopping(monitor='val_custom_f1', mode='max', patience=50, 
                                               restore_best_weights=True, verbose=2)

filepath = "/content/gdrive/My Drive/epochs:{epoch:03d}-val_f1:{val_custom_f1:.3f}.hdf5"

checkpoint_callback = ModelCheckpoint(filepath= filepath, monitor='val_custom_f1', mode='max',
                                                         save_weights_only=True, save_best_only=True)


deeper_history = deeper_model.fit(train_generator,
                              validation_data = validation_generator,
                              initial_epoch=100,
                              epochs=200,
                              batch_size=64,
                              callbacks=[early_stopping, checkpoint_callback]) 

Epoch 101/200
244/244 [==============================] - 342s 1s/step - loss: 0.3655 - accuracy: 0.8707 - custom_f1: 0.8679 - custom_recall: 0.8399 - custom_precision: 0.8989 - val_loss: 0.4391 - val_accuracy: 0.8441 - val_custom_f1: 0.8384 - val_custom_recall: 0.8152 - val_custom_precision: 0.8646
Epoch 102/200
244/244 [==============================] - 321s 1s/step - loss: 0.3585 - accuracy: 0.8749 - custom_f1: 0.8737 - custom_recall: 0.8500 - custom_precision: 0.8996 - val_loss: 0.4716 - val_accuracy: 0.8472 - val_custom_f1: 0.8461 - val_custom_recall: 0.8254 - val_custom_precision: 0.8689
Epoch 103/200
244/244 [==============================] - 320s 1s/step - loss: 0.3666 - accuracy: 0.8722 - custom_f1: 0.8717 - custom_recall: 0.8504 - custom_precision: 0.8947 - val_loss: 0.4718 - val_accuracy: 0.8395 - val_custom_f1: 0.8341 - val_custom_recall: 0.8112 - val_custom_precision: 0.8599
Epoch 104/200
244/244 [==============================] - 319s 1s/step - loss: 0.3569 - accuracy: 0.8

In [ ]:
deeper_model.load_weights('/content/gdrive/My Drive/epochs:145-val_f1:0.866.hdf5')